# Příprava dat a jejich popisná charakteristika

## Autoři:
    - Vojtěch Kulíšek
    - Lukáš Plevač    
    - Pavel Šesták

## Zadání
Z dostupných datových sad si zvolte jednu datovou sadu, kterou se budete dále zabývat. Stáhněte si zvolenou datovou sadu z uvedeného zdroje a prostudujte si dostupné informace k této datové sadě.
Proveďte explorativní analýzu zvolené datové sady. Pro každý následující bod implementujte odpovídající sekci ve zdrojovém kódu a zjištěné výsledky popište v dokumentaci:
prozkoumejte jednotlivé atributy datové sady, jejich typ a hodnoty, kterých nabývají (počet hodnot, nejčastější hodnoty, rozsah hodnot atd.)
prozkoumejte rozložení hodnot jednotlivých atributů pomocí vhodných grafů, zaměřte se i na to, jak hodnota jednoho či dvou atributů ovlivní rozložení hodnot jiného atributu. Do dokumentace vložte alespoň 5 různých grafů, zobrazujících zjištěná rozložení hodnot. Použijte různé typy grafů (např. bodový graf, histogram, krabicový nebo houslový graf, graf složený z více podgrafů apod.).
zjistěte, zda zvolená datová sada obsahuje nějaké odlehlé hodnoty.
proveďte podrobnou analýzu chybějící hodnot (celkový počet chybějících hodnot, počet objektů s více chybějícími hodnotami atd.).
proveďte korelační analýzu numerických atributů (k analýze využijte i grafy a korelační koeficienty).
Připravte 2 varianty datové sady vhodné pro dolovací algoritmy. Můžete uvažovat dolovací úlohu uvedenou u datové sady nebo navrhnout vlastní dolovací úlohy. V případě vlastní dolovací úlohy ji specifikujte v dokumentaci. V rámci přípravy datové sady proveďte následující kroky:
Odstraňte z datové sady atributy, které jsou pro danou dolovací úlohu irelevantní.
Vypořádejte se s chybějícími hodnotami. Pro odstranění těchto hodnot využijte alespoň dvě různé metody pro odstranění chybějících hodnot.
Vypořádejte se s odlehlými hodnotami, jsou-li v datové sadě přítomny.
Pro jednu variantu datové sady proveďte diskretizaci numerických atributů tak, aby výsledná datová sada byla vhodná pro algoritmy, které vyžadují na vstupu kategorické atributy.
Pro druhou variantu datové sady proveďte vhodnou transformaci kategorických atributů na numerické atributy. Dále pak proveďte normalizaci numerických atributů, které má smysl normalizovat. Výsledná datová sada by měla být vhodná pro metody vyžadující numerické vstupy.

In [ ]:
import subprocess
import sys
import os
requirementsPath = os.path.join(os.path.dirname(os.path.realpath('__file__')),"requirements.txt")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", requirementsPath])
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from sklearn.linear_model import LinearRegression
import re
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

PLOT_GRAPHS = True
PLOT_STATS = True

## Explorační analýza
V této části se blíže seznámíme s daty, které máme dále upravovat. V rámcí modelu CRISP-DM jsme v sekci pochopení dat. Pro tuto úlohu jsme si zvolili datovou sadu z průzkumu platů v IT sektoru z roků 2018 až 2020. Dále budeme pracovat pouze s nejnovějšími daty z roku 2020 jelikož ekonomická situace je v dnešní době velmi dynamická a už tak se jedná o stará data. Starší data můžeme dále použít pro validaci našich klasifikátorů a porovnat jak moc se datové sady vzájemně liší.

### Načtení datových souborů
V této sekci si nahrajeme zvolený datový soubor do operační paměti pomocí knihovny pandas. Z datové odstraníme atributy, které zjevně nepocházejí od uživatele jako je například časová značka. Jelikož jsou to data z dotazníku, tak některé otázky jsou rozsáhle popsány, aby uživatel věděl co přesně má vyplnit, pro naše účely si tyto sloupce přejmenujeme, aby se s daty dále lépe pracovalo.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


FILES = ["data/IT_Salary_Survey_EU_2018.csv", "data/IT_Salary_Survey_EU_2019.csv", "data/IT_Salary_Survey EU_2020.csv"]

data = pd.read_csv(FILES[2])

if "Timestamp" in data:
    data.drop(["Timestamp"], axis=1, inplace=True)

if "Zeitstempel" in data:
    data.drop(["Zeitstempel"], axis=1, inplace=True)

if "0" in data:
    data.drop(["0"], axis=1, inplace=True)

data.rename(columns = {
    "Your main technology / programming language":'Main Technology',
    "Other technologies/programming languages you use often" : "Other technologies",
    "Yearly brutto salary (without bonus and stocks) in EUR": "Yearly brutto",
    "Annual bonus+stocks one year ago. Only answer if staying in same country" : "Bonus and stocks in same country",
    "Have you lost your job due to the coronavirus outbreak?" : "Job lost due covid",
    "Have you received additional monetary support from your employer due to Work From Home? If yes, how much in 2020 in EUR" : "Home office compensation",
    "Position " : "Position",
    }, inplace = True)

data.sample(10)

In [ ]:
#pd.plotting.parallel_coordinates(data, "Your level")
#plt.show()
#note1 i found whole line with NaN filter it

In [ ]:
def plot_graphs(data: pd.DataFrame) -> None:
    """
    Get pandas dataframe. Describe and plot graphs for all columns in dataframe.
    
    PRE CONDITION: If you want just text info about params specify PLOT_STATS. If you want pyplot graphs as output define constant PLOT_GRAPHS to True.
    """
    if not PLOT_STATS and not PLOT_GRAPHS:
        return
    columns = data.columns

    dtypes = data.dtypes

    for column in columns:

        print(data[column].describe())
        
        if not PLOT_GRAPHS:
            continue

        figure, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
        figure.suptitle("Data for: "+ column, fontsize=15)
        axes[0].set_title("NaN values", fontsize=12)

        IsNan = data[column].isna().sum()
        IsNotNan = len(data[column])-IsNan
        axes[0].bar("Unfilled", IsNan)
        axes[0].bar("Filled", IsNotNan)
        axes[1].set_title("Data distribution", fontsize=12)

        if dtypes[column] == "object":
            data[column].value_counts().plot(kind='bar')
        elif dtypes[column] == "float64":
            data[column].plot(kind='box')

        plt.show()

plot_graphs(data)

Z vygenerovaných grafů je vidět, že většina dat je vyplněná. V případě, že chybí v jistém sloupci mnoho dat tak se jedná většinou o věci jako bonusy, různé kompenzace a podobně. U těchto atributů se dá předpokládat, že když například uživatel nedostal kompenzaci za práci z domu tak položku nevyplnil. Pro další práci s datovou sadou si podrobně projdeme datové sloupce a zjistíme jaké konkrétní chyby se vyskytují v datové sadě, aby jsme se na ně mohli soustředit v další části čištění dat.

#### Age
Většina hodnot je vyplněná. Dokonce jsou i v rozumném rozpětí, kde většina respondentů je ve věku 30 až 35 let. Jedná se o numerický atribut.

#### Gender
Pohlaví také většina respondentů vyplnila a je rozděleno do tří kategorií, zde nebude nutné nějaké významné čištění, jen doplníme chybějící hodnoty nejčastější hodnotou. Jelikož se jedná o datovou sadu z technického oboru, tak není překvapující, že rozložení pohlaví je výrazně nevyvážené.

#### City
Tento kategorický atribut bude potřeba pro další práci nějak shluknout. Podle hodnot je zřejmé, že sběr dat pochází z Německa.

#### Position
Tento kategorický atribut bude také pro další práci potřeba vyčistit, konkrétně odfiltrovat hodnoty s nízkou četností.

#### Total years of experience
Tento zřejmě numerický atribut byl knihovnou pandas interpretován jako kategorický z důvodu pár textových odpovědí. Pro naši další práci bude potřeba převést na numerický a textové odpovědi vyfiltrovat.

#### Years of experience in Germany
Stejný případ jako u Total years of experience.

#### Seniority level
Kategorický atribut, který se zjevně rozpadá do čtyř kategorií a dále mnoho odlehlých hodnot, které můžeme nejspíše shluknout.

#### Main Technology
Pro nás se jedná o velmi významný sloupec, bohužel je velmi nešťastně zadaný. Pro naši další práci rozdělíme řetězce na jednotlivé technologie a zobrazíme si jejich histogram, z kterého vyčteme nejvíce používané technologie.

#### Other technologies
Obdobný problém jako u main technology jen zde se uživatelé rozepsali ještě více a zde se jedná v této formě již o úplně nepoužitelný atribut

#### Yearly brutto
Numerický atribut, z kterého zatím bohužel nic nevyčteme jelikož obsahuje dost výraznou anomálii, které se před další prací budeme muset zbavit.

#### Yearly bonus + stocks in EUR
Zjevně numerický atribut, který je interpretován jako kategorický. Bude nutné převést na numerický. Mnoho nezadaných hodnot, ale z podstaty věci se dá předpokládat, že když někdo nevyplní bonus tak žádný nemá.

#### Annual brutto salary one year ago.
Roční plat před jedním rokem, opět graf zatížen odlehlou hodnotou, kterou bude potřeba vyfiltrovat.

#### Bonus and stocks in same country
Obdobně jako u Yearly bonus + stocks in EUR.

#### Number of vacation days
Bude nutné převést na numerický atribut, jelikož naše datová sada obsahuje většinu lidí co jsou zaměstnanci tak chybějící hodnoty budeme doplňovat střední hodnotou.

#### Employment status
Většina respondentů jsou zaměstnanci na plný úvazek. Můžeme pouze očistit o málo frekventované kategorie, zde jich ale naštěstí není tolik.

#### Contract duration
Většina smluv je na dobu neurčitou. Vydíme jednu odlehlou hodnotu 0, které by bylo vhodné se zbavit a pár chybějících hodnot, které si můžeme v tomhle sloupci dovolit nahradit nejčetnější hodnotou, jelikož jasně převládá a chybějících hodnot je málo.

#### Main language at work
Kategorický atribut, kde převládají dvě hodnoty. Zbytek je možno rozdělit separátorem a případně oddělat nesmyslné hodnoty.

#### Company size
Nezašumělý kategorický atribut, kde pár chybějících hodnot můžeme nahradit nejčetnější hodnotou.

#### Company type
Mnoho hodnot v tomto kategorickém atributu má malý výskyt. Dominují zde tři kategorie a zbytek bude vhodné shluknout.

#### Job lost due covid
Zjevně binární atribut, který obsahuje mnoho slovních odpovědí, bude nutné očistit.

#### Have you been forced to have a shorter working week
Mnoho chybějících hodnot, šlo by z tohoto vyčíst průměrnou dobu v práci, proto chybějící hodnoty nahradíme 40ti hodinami jako standardní pracovní týden, což je asi výchozí hodnota a člověk který nemá zkrácený pracovní úvazek tuto hodnotu zřejmě ignoroval.

#### Home office compensation
Další zjevně numerický atribut, který bude potřeba očistit od slovních hodnot.


### Korelační analýza
korelační analýza slouží pro hledání podobností mezi atributy. Když najdeme silnější korelaci mezi atributy, tak můžeme využít regresi pro dopočítání chybějících hodnot mezi takto korelovanými atributy a daný odhad bude daleko přesnější než například medián ze souboru. Korelační analýza pracuje nad numerickými atributy. Jak jsme si ukázali při analýze datových atributů tak mnoho numerických je interpretováno jako kategorické atributy z důvodu například nějaké textové odpovědi. Takto detekované atributy si pomocí knihovny pandas převedeme na numerické, kde špatné hodnoty převedeme na NaN.

In [ ]:
#
data['Total years of experience'] = pd.to_numeric(data['Total years of experience'], errors="coerce")
data['Age'] = pd.to_numeric(data['Age'], errors="coerce")
data['Years of experience in Germany'] = pd.to_numeric(data['Years of experience in Germany'], errors="coerce")
data['Yearly bonus + stocks in EUR'] = pd.to_numeric(data['Yearly bonus + stocks in EUR'], errors="coerce")
data['Bonus and stocks in same country'] = pd.to_numeric(data['Bonus and stocks in same country'], errors="coerce")
data['Number of vacation days'] = pd.to_numeric(data['Number of vacation days'], errors="coerce")
data['Home office compensation'] = pd.to_numeric(data['Home office compensation'], errors="coerce")

data.sample(5)

In [ ]:
cov_matrix = data.corr('spearman')#pd.DataFrame.corr(data)
#print(cov_matrix)
sn.heatmap(cov_matrix, annot=True)
plt.show()

Z heat mapy můžeme vidět závislosti mezi věkem a odpracovanými roky což je pochopitelné a tuto závislost tedy budeme moci využít.
Další silná závislost je mezi sloupci ohledně investic, které takto dopočítávat ale nebudeme.

## Přípravy datové sady
Máme za sebou explorativní analýzu, v které jsme identifikovali jisté nedostatky v datové sadě. Nyní se pokusíme data očistit a doplnit.
### Ořezání prázdných záznamů a odfiltrování neužitečných atributů
Zde odstraníme neužitečné atributy a zahodíme záznamy, které obsahují míň jak šest vyplněných hodnot, jelikož takové záznamy mají malou přídanou hodnotu pro datovou sadu a mnoho dat by bylo bráno jako průměr či nějak korelováno. 


In [ ]:
data2 = data
if "Timestamp" in data.columns:
    data2 = data.drop('Timestamp', axis=1)

if 'Are you getting any Stock Options?' in data2.columns:
    data2 = data2.drop('Are you getting any Stock Options?', axis=1)


print("Records: ",len(data2))
data2 = data2.dropna(thresh=6)

print("Records: ",len(data2))

### Dopočítání chybějících hodnot
V této části se pokusíme doplnit do datové sady chybějící hodnoty pomocí různých přístupů. Kde půjde odhadnout hodnotu na základě vysoké korelace s jiným atributem tak využijeme regresi. U sloupců, kde to dává smysl tak použijeme střední hodnotu. Jsou sloupce kde můžeme očekávat, že není vyplněno jelikož se to daného respondenta netýkalo (například chybějící položka bonusy ve firmě, tak zřejmě žádné bonusy nemá).


#### Tvorba regresních prediktorů
Zde vytvoříme prediktory pro korelované atributy. Jako trénovací data vezmeme všechny záznamy bez chybějících hodnot.

In [ ]:

#convert object types to float, errors coerce specified if its not a numeric type replace with NaN
data2['Total years of experience'] = pd.to_numeric(data2['Total years of experience'], errors="coerce")
data2['Age'] = pd.to_numeric(data2['Age'], errors="coerce")

dataLearn = data2.dropna()

AgePredict = LinearRegression()
AgePredict = AgePredict.fit(dataLearn[['Total years of experience']].values, dataLearn[['Age']].values)

YearOfExperiencePredict = LinearRegression()
YearOfExperiencePredict = YearOfExperiencePredict.fit(dataLearn[['Age']].values, dataLearn[['Total years of experience']].values)


YearOfExperienceInGermanyPredict = LinearRegression()
YearOfExperienceInGermanyPredict = YearOfExperienceInGermanyPredict.fit(dataLearn[['Total years of experience']].values, dataLearn[['Years of experience in Germany']].values)



YearScore = YearOfExperiencePredict.score(dataLearn[['Age']].values, dataLearn[['Total years of experience']].values)
print(YearScore)

x = np.linspace(0, 30, 30)
plt.plot(dataLearn['Total years of experience'], dataLearn['Age'], 'o')

AgeScore = AgePredict.score(dataLearn[['Total years of experience']].values, dataLearn[['Age']].values)
print(AgeScore)

diabetes_y_pred = AgePredict.predict(np.array([x]).T)


plt.plot(x, diabetes_y_pred, color="red", linewidth=3)

In [ ]:
salaryColumnName = ""

if "Current Salary" in dataLearn:
    salaryColumnName = "Current Salary"
elif "Yearly brutto" in dataLearn:
    salaryColumnName = "Yearly brutto"

AgePredictDependsOnSalary = LinearRegression()
AgePredictDependsOnSalary = AgePredictDependsOnSalary.fit(dataLearn[[salaryColumnName]].values, dataLearn[['Age']].values)

x = np.linspace(0, 175000, 175000)
plt.plot(dataLearn[salaryColumnName], dataLearn['Age'], 'o')

AgePredictDependsOnSalary.score(dataLearn[[salaryColumnName]].values, dataLearn[['Age']].values)


diabetes_y_pred = AgePredictDependsOnSalary.predict(np.array([x]).T)
plt.plot(x, diabetes_y_pred, color="red", linewidth=3)

Tento prediktor se snažil naučit vztah mezi věkem a platem. Jak vidíme tak zde nějak silný vztah neexistuje. Použití takového prediktoru by vedlo k chybám.

### Doplnění nekorelovaných atributů

In [ ]:
data2.loc[data2['Yearly bonus + stocks in EUR'].isna(), 'Yearly bonus + stocks in EUR'] = 0
data2.loc[data2['Bonus and stocks in same country'].isna(), 'Bonus and stocks in same country'] = 0
data2.loc[data2['Number of vacation days'].isna(), 'Number of vacation days'] = data2['Number of vacation days'].mode()[0]
data2.loc[data2['Job lost due covid'].isna(), 'Job lost due covid'] = "No"
data2.loc[data2['Home office compensation'].isna(), 'Home office compensation'] = 0
data2.loc[data2['Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week'].isna(), 'Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week'] = 40 #standard hours per week
data2.loc[data2['Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week'] == 0, 'Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week'] = 40 #standard hours per week
data2.loc[data2['Gender'].isna(), 'Gender'] = data2['Gender'].mode()[0]
data2.loc[data2['City'].isna(), 'City'] = data2['City'].mode()[0]
data2.loc[data2['Seniority level'].isna(), 'Seniority level'] = data2['Seniority level'].mode()[0]
data2.loc[data2['Main language at work'].isna(), 'Main language at work'] = data2['Main language at work'].mode()[0]
data2.loc[data2['Company size'].isna(), 'Company size'] = data2['Company size'].mode()[0]
data2.loc[data2['Company type'].isna(), 'Company type'] = data2['Company type'].mode()[0]


data2.loc[data2['Employment status'].isna(), 'Employment status'] = data2['Employment status'].mode()[0]
data2.loc[data2['Сontract duration'].isna(), 'Сontract duration'] = data2['Сontract duration'].mode()[0]

data2.drop(data2.loc[data2["Main Technology"].isna()].index, inplace=True)
data2.drop(data2.loc[data2["Other technologies"].isna()].index, inplace=True)



### Doplnění korelovaných atributů pomocí prediktorů

In [ ]:
data2.loc[data2['Age'].isna() & data2['Total years of experience'].notna(), 'Age'] = \
data2.loc[data2['Age'].isna() & data2['Total years of experience'].notna(), 'Total years of experience'].apply(lambda exp : np.round(AgePredict.predict([[exp]]))[0][0])

data2 = data2.drop(data2.loc[data2['Age'].isna() & data2['Total years of experience'].isna()].index)

In [ ]:
data2.loc[data2['Years of experience in Germany'].isna() & data2['Total years of experience'].notna(), 'Years of experience in Germany'] = \
data2.loc[data2['Years of experience in Germany'].isna() & data2['Total years of experience'].notna(), 'Total years of experience'].apply(lambda exp : np.round(YearOfExperienceInGermanyPredict.predict([[exp]]))[0][0])

data2 = data2.drop(data2.loc[data2['Years of experience in Germany'].isna() & data2['Total years of experience'].isna()].index)

#### agregace pozice
Zde se snažíme snížit počet různých kategorií pro pozici

In [ ]:
originalLenght = len(data2['Position'].unique())

for i in data2.index:
    if not pd.isnull(data2['Position'][i]):
        data2.loc[i, 'Position'] = re.sub("(\s*senior\s*|\s*junior\s*|\s*middle\s*)", " ", data2['Position'][i].lower())
        data2.loc[i, 'Position'] = re.sub("(^\s*|\s*$)", " ", data2['Position'][i])
        data2.loc[i, 'Position'] = re.sub("\s+", " ", data2['Position'][i])

data2.drop(data2.loc[data2['Position'].isna()].index, inplace=True)        
NewLenght = len(data2['Position'].unique())
print("Reduced", originalLenght-NewLenght)

In [ ]:

data2.loc[data2['Total years of experience'].isna() & data2['Age'].notna(), 'Total years of experience'] = \
data2.loc[data2['Total years of experience'].isna() & data2['Age'].notna(), 'Age'].apply(lambda age : np.round(YearOfExperiencePredict.predict([[age]]))[0][0])

data2.drop(data2.loc[data2['Total years of experience'].isna() & data2['Age'].isna()].index, inplace=True)


#### Doplnění platu
Zde nejspíše člověk neuvedl druhý plat, jelikož se mu plat nezměnil a přišlo mu to  zbytečné. Můžeme křížově doplnit.

In [ ]:

originalLenght = len(data2)


currentSalary = "Yearly brutto"
salaryOneYearAgo = "Annual brutto salary (without bonus and stocks) one year ago. Only answer if staying in the same country"


data2.loc[data2[currentSalary].isna() & data2[salaryOneYearAgo].notna(), currentSalary] = data2.loc[data2[currentSalary].isna() & data2[salaryOneYearAgo].notna(), salaryOneYearAgo]
data2.loc[data2[salaryOneYearAgo].isna() & data2[currentSalary].notna(), salaryOneYearAgo] = data2.loc[data2[salaryOneYearAgo].isna() & data2[currentSalary].notna(), currentSalary]

data2.drop(data2.loc[data2[currentSalary].isna() & data2[salaryOneYearAgo].isna()].index, inplace=True)


"""



salaries = {
    'Current Salary': ['Salary one year ago', 'Salary two years ago'],
    'Salary one year ago': ['Salary two years ago', 'Current Salary'],
    'Salary two years ago': ['Salary one year ago', 'Current Salary']
}

for i in data2.index:
    for j in salaries.keys():
        if pd.isnull(data2[j][i]):
            for salary in salaries[j]:
                if not pd.isnull(data2[salary][i]):
                    data2.loc[i, j] = data2[salary][i]
                    break

    if pd.isnull(data2['Current Salary'][i]):
        data2.drop(i, inplace=True)

"""


NewLenght = len(data2)
print("Reduced", originalLenght-NewLenght)

In [ ]:
data2.Position.str.split(expand=True).stack().value_counts()
'''
print(x.keys()[0], x[0])

dataLearn = data2.dropna()

AgePredict = LinearRegression()
AgePredict = AgePredict.fit(x.keys()[0], dataLearn[['Age']].values)

x = np.linspace(0, 30, 30)
plt.plot(dataLearn['Position'], dataLearn['Age'], 'o')

AgeScore = AgePredict.score(x.keys()[0], dataLearn[['Age']].values)
print(AgeScore)

diabetes_y_pred = AgePredict.predict(np.array([x]).T)
plt.plot(x, diabetes_y_pred, color="red", linewidth=3)
'''

### Detekce anomalií
V explorativní analýze jsme narazili na některé nečitelné numerické grafy, jelikož měřítko na ose výrazně ovlivnila odlehlá hodnota. Nyní se takovýchto hodnot pokusíme zbavit pomocí metody zscore. ta předpokládá normální rozdělení. V našem okolí se většina jevů chová podle normálního rozdělení, což nám blíže říká centrální limitní věta. V normálním rozdělení 99.7% hodnot leží v intervalu <-3 σ; 3 σ> a oproto hodnoty mimo tento interval odstraníme. 

In [ ]:
originalLenght = len(data2)


#print("dtypes: ", data.dtypes)
print("columns: ", data2.select_dtypes(include=["number"]).columns)

def drop_numerical_outliers(df, z_thresh=3):
    # Constrains will contain `True` or `False` depending on if it is a value below the threshold.
    constrains = df.select_dtypes(include=["number"]) \
        .apply(lambda x: np.abs(stats.zscore(x)) < z_thresh) \
        .all(axis=1)
    # Drop (inplace) values set to be rejected
    df.drop(df.index[~constrains], inplace=True)

drop_numerical_outliers(data2)



NewLenght = len(data2)
print("Reduced: ", originalLenght-NewLenght)
data2.head()

#### Výpis nejpoužívanějších technologií
V explorativní analýze jsme detekovali, že mnoho lidí zadalo více technologií. Nyní se pokusíme hodnoty rozdělit a vykreslit graf četností použití technologií.

In [ ]:
#main technologies

mainTechs = data2["Main Technology"].str.split(pat=r"[,\/\s]+",expand=True).stack().value_counts()

outliers = mainTechs.loc[mainTechs <= 1]
outliersCount = mainTechs.loc[mainTechs <= 1].sum()

del mainTechs['']

plt.rcParams["figure.figsize"] = (20,10)
mainTechs = mainTechs.drop(mainTechs.loc[mainTechs.isin(outliers)].index)
mainTechs["Other"] = outliersCount

mainTechs.plot(kind='bar')
plt.xticks(rotation=90)

In [ ]:
#other technologies

otherTechs = data2["Other technologies"].str.lower().str.split(pat=r"[,\/\s]+",expand=True).stack().value_counts()

outliers = otherTechs.loc[otherTechs <= 1]
outliersCount = otherTechs.loc[otherTechs <= 1].sum()

del otherTechs['']

otherTechs = otherTechs.drop(otherTechs.loc[otherTechs.isin(outliers)].index)
otherTechs["Other"] = outliersCount

otherTechs.plot(kind='bar')
plt.xticks(rotation=90)

### Kontrola po čištění
Nyní si zobrazíme očištěná data. Grafy by měli být čitelnější než v explorativní analýze.

In [ ]:
plot_graphs(data2)

## Predikce platu na základě ostatních parametrů
### Převod dat pro data miningové úlohy
V této sekci si připravíme dvě datové sady, jedna bude obsahovat pouze numerické atributy a druhé pouze kategorické.
Pro převod kategorického atributu na numerický použijeme kódování one hot. Tato metoda vytvoří mnoho sloupců v závislosti na počtu kategorických atributů a počtu různých hodnot. 
Toto kódování je vhodné pro strojové učení a například v našem případě se odhad pomocí regrese zlepšil díky tomuto kódování z 85% na 94%.

V opačném případě, když budeme připravovat kategorickou sadu, tak musíme převést numerický atribut na kateogorie. Použijeme qcut, kde pomocí kvantilů rozdělíme na určitý počet košů. Mnoho atributů jako napřiklad bonusy mají většinu hodnot rovnou nule. Zde by nešlo použit mnoho košů. Naopak jsou atributy, které klidně můžeme rozdělit do více košů jako například věk. Proto začneme u každého atributz s větším počtem košů a budeme snižovat dokud se nám nepodaří převést (Každý koš musí mít vlastní hodnotu kvantilu).

In [ ]:
nData = data2.copy(deep=True)

nData["Other technologies"] = data2["Other technologies"].str.split(pat=r"[,\/\s]+").str.len()

nData.head()

In [ ]:
attributes = ["Gender", "City", "Position", "Seniority level", "Main Technology", "Employment status", "Сontract duration", "Main language at work", "Company size", "Company type", "Job lost due covid", "Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week", "Home office compensation"]

one_hot = pd.get_dummies(nData)

nData = one_hot

nData=(nData-nData.min())/(nData.max()-nData.min())

#for attr in attributes:
#    pass
#    nData[attr] = pd.Categorical(nData[attr])
#    nData[attr] = nData[attr].cat.codes


#one_hot.head()

nData.head()

### Prediction

In [ ]:
def svm(dataset, target):
    brutto = dataset[target]
    testRegrData = dataset.drop([target], axis=1)


    bruttoPredict = LinearRegression()
    bruttoPredict = bruttoPredict.fit(testRegrData.values, brutto)

    #x = np.linspace(0, 175000, 175000)
    #plt.plot(dataLearn[salaryColumnName], dataLearn['Age'], 'o')

    accDif = 0.0

    for i in range(len(testRegrData)):
        p = bruttoPredict.predict([testRegrData.iloc[i]])
        br = brutto.iloc[i]
        accDif += np.abs(p - br) 

    accDif /= len(testRegrData)

    print("Average delta: ", accDif)

    sc = bruttoPredict.score(testRegrData, brutto)
    print("Accuraci: ", sc)

svm(nData, "Yearly brutto")

### Data to cat

In [ ]:
cData = data2.copy(deep=True)

attributes = ["Age", "Total years of experience", "Years of experience in Germany", "Yearly brutto", "Yearly bonus + stocks in EUR", "Annual brutto salary (without bonus and stocks) one year ago. Only answer if staying in the same country", "Bonus and stocks in same country", "Number of vacation days", "Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week", "Home office compensation"]

for attr in attributes:
    #print("attr: ", attr)
    #print("attr: ", cData[attr].describe())

    #cData[attr] = pd.qcut(cData[attr], q=1) #TODO... increase q need normalize  
    
    for q in reversed(range(5)):
        try:
            cData[attr] = pd.qcut(cData[attr], q=q).astype(str)
            print("attr: ", attr, " with q: ", q)
            break
        except:
            continue
        

#pd.cut
#PD.CUT(column, bins=[ ],labels=[ ])
#pd.cut(df.Age,bins=[0,2,17,65,99],labels=[‘Toddler/Baby’,’Child’,’Adult’,’Elderly’])
#print()

#cData.head()